# 常规装饰器（函数实现）

In [4]:
from typing import Callable
from functools import wraps
def decorator(fn: Callable):
    @wraps(fn)  # 指明下面的函数是用来包裹fn的函数，如果不加这个，则原函数加上@decorator后将直接被wrapper函数替换，simple_plus.__name__会变成wrapper
    def wrapper(*args,**kwargs):
        fn_name = fn.__name__
        print(f'{fn_name} start')
        result = fn(*args,**kwargs)
        print(f'{fn_name} end')
        return result
    return wrapper

@decorator # 本质上就是将simple_plus替换为decorator(simple_plus)
def simple_plus(a,b):
    return a+b
print(simple_plus.__name__) # 确认原函数依然是原函数，因为@wraps的作用
result = simple_plus('1','2')
print(result)

simple_plus
simple_plus start
simple_plus end
12


# 本身也带参数的装饰器
外面再包一层带参数的，里面都不用变

In [6]:
def welcome(name):
    def decorator(fn: Callable):
        @wraps(fn)
        def wrapper(*args,**kwargs):
            print(f'Welcome {name}')
            print(f'{fn.__name__} start')
            result = fn(*args,**kwargs)
            print(f'{fn.__name__} end')
            return result
        return wrapper
    return decorator

@welcome('User')
def simple_sum(*args):
    return sum(args)

print(simple_sum(1,2,3,4))

Welcome User
simple_sum start
simple_sum end
10


# 生成器
当一个函数包含yield语句后，就不再是一个普通的函数，而是一个生成器

return只能执行一次，yield可以执行好多次

In [28]:
def hello():    # 执行时返回一个可迭代的对象
    print('Step 1:')
    yield 1
    print('Step 2:')
    yield 2
    print('Step 3:')
    yield 3
a = hello() # a是可迭代对象
print(next(a))    # 测试一下a的迭代方法，输出Step 1:1。每次调用next时，函数只会执行到下一个yield语句
for i in a: # for循环迭代a的剩余部分（后台实际上也是在执行next方法）分别输出Step 2:2 Step 3:3
    print(i)

Step 1:
1
Step 2:
2
Step 3:
3


相比于在函数中构造出一个大的list并最后return，使用yield可以大大节省资源

## 实战：构造一个生成从0到n-1的每个数的平方值的生成器

In [4]:
def square(n):
    for i in range(n):
        yield i**2

for i in square(4):
    print(i)

# 不使用生成器的版本
# def square_wo_generator(n):
#     result = []
#     for i in range(n):
#         result.append(i**2)
#     return result
# for i in square_wo_generator(4):
#     print(i)

0
1
4
9
0
1
4
9


## 上下文管理器

一个上下文管理器是一个对象，它定义了运行时的上下文，使用with语句来执行
### with语句
```py
with context as ctx：
    # 使用这个上下文对象
# 上下文对象已经被清除了
```

# 面向对象的python

类本身是type类型的对象

In [12]:
class Student:
    pass
xm = Student()
print(type(xm))
# 类本身是type类型的对象
print(type(Student))

<class '__main__.Student'>
<class 'type'>


## 类变量
属于类本身这个对象的属性

所有该类的对象都共享类变量。类似于Java的静态变量。但是python的静态变量是别的概念

In [11]:
class Student:
    student_count = 8

print(Student.student_count)    # 直接取值
print(getattr(Student,'student_count')) # 用getattr取值
# print(Student.unknown)
print(getattr(Student,"unknown","10"))

Student.student_count = 89
setattr(Student,'student_count',100)
print(Student.student_count)

Student.newattribute = "hello"
print(Student.newattribute) # python是动态语言

8
8
10
100
hello


setattr相当于Java的反射，动态修改类

### 删除类变量

In [12]:

print(Student.newattribute)
print(Student.student_count)

del Student.newattribute
delattr(Student,'student_count')

print(Student.newattribute)
print(Student.student_count)

hello
100


AttributeError: type object 'Student' has no attribute 'newattribute'

In [13]:
s1 = Student()
s2 = Student()
Student.student_count = 4
print(s1.student_count)
print(s2.student_count)

4
4


### 类变量的存储
类变量全部存放在类变量__dict__这个字典中，但不要直接修改__dict__的内容

In [15]:
from pprint import pprint
pprint(Student.__dict__)

mappingproxy({'__dict__': <attribute '__dict__' of 'Student' objects>,
              '__doc__': None,
              '__module__': '__main__',
              '__weakref__': <attribute '__weakref__' of 'Student' objects>,
              'student_count': 4})


## 实例变量与函数

定义实例函数

In [20]:
class Student:
    def say_hello(self,msg):
        print(f"hello {msg}")
a = Student()
a.say_hello('Mike')

hello Mike


### 认识__init__函数
1. 在内存中创建了物理对象
2. 调用__init__方法初始化刚刚创建的对象

In [27]:
class Student:
    def __init__(self,name):
        self.name = name
    def say_hello(self,msg):
        print(f"hello {msg}, my name is {self.name}")
s1 = Student('Jack')
print(s1.name)
s1.say_hello('Mike')

Jack
hello Mike, my name is Jack


直接给对象加属性

In [28]:
s1.gender = 'Male'
print(s1.gender)

Male


## 私有属性与函数

### 私有属性与函数的用途
防止它们在类的外部被使用

在Python中并没有严格的权限限定符去进行限制，主要通过命名来进行区分

### 如何定义私有属性与函数
通过给属性和函数名称添加_或__前缀

In [31]:
class Student:
    def __init__(self,name,gender):
        self._name = name
        self.__gender = gender
    def _change_gender():
        pass
    def __change_name():
        pass
    def say_hello(self,msg):
        print(f'Hello {msg}, my name is {self._name}')
s1 = Student('Jack','Male')
print(s1._name) # 尝试获取一个下划线前缀的属性不会报错，但是不符合规范
# print(s1.__gender)  # 在外部尝试获取两个下划线前缀的属性会报错
print(s1._Student__gender)  # 如果在外部非要用得这样写，在前面加上一个下划线和类名

Jack
Male


## 类方法和静态方法

### 类方法
类方法需要用@classmethod装饰器来定义

类方法的第一个参数是类本身

In [33]:
class Student:
    name = 'Student'
    @classmethod
    def get_instance(cls):
        return cls()
    @classmethod
    def hello(cls):
        print(cls.name)
student = Student.get_instance()
Student.hello()

Student


### 静态方法
静态方法使用@staticmethod装饰器来定义

静态方法只是定义在类范围内的一个函数而已。通常用来定义一些工具方法

In [34]:
class Student:
    name = 'Student'
    @staticmethod
    def out():
        print(f"hello {Student.name}")  # 也能用类的属性，但不用cls，得用类名
Student.out()

hello Student


## 常用的特殊方法

双下划线开头，双下划线结尾

### 特殊方法__str__
返回一个描述对象本身的字符串，该描述主要面对用户。当调用str()函数时就会返回__str__的值

In [35]:
class MyDate:
    def __init__(self,year,month,day):
        self.year = year
        self.month = month
        self.day = day
    def __str__(self):
        return f"{self.year}-{self.month}-{self.day}"
    
my_date = MyDate(2022,11,3)
print(str(my_date))

2022-11-3


### 特殊方法__repr__
用于返回一个描述对象本身的字符串

该描述的主要目标是机器或者开发者，通常比__str__包含更丰富的调试信息

使用repr()函数时返回

和__str__的区别就是在于用途

In [36]:
class MyDate:
    def __init__(self,year,month,day):
        self.year = year
        self.month = month
        self.day = day
    def __repr__(self):
        return f"MyDate:{self.year}-{self.month}-{self.day}"

my_date = MyDate(2022,11,3)
print(repr(my_date))

MyDate:2022-11-3


### 特殊方法__eq__
用于实现对比两个对象是否相等的逻辑

调用==时使用

In [38]:
my_date1 = MyDate(2022,11,3)
my_date2 = MyDate(2022,11,3)
print(my_date1 is my_date2) # is比较的是地址
print(my_date1 == my_date2) # ==根据__eq__方法比较值是否相等，但如果没有重写过__eq__方法，就没用


False
False


In [39]:
class MyDate:
    def __init__(self,year,month,day):
        self.year = year
        self.month = month
        self.day = day
    def __eq__(self,other): # other是要比较的对象
        if not isinstance(other, MyDate):
            return False
        return self.year == other.year and self.month == other.month and self.day == other.day
my_date1 = MyDate(2022,11,3)
my_date2 = MyDate(2022,11,3)
print(my_date1 is my_date2) # is比较的是地址
print(my_date1 == my_date2) # ==根据__eq__方法比较值是否相等，但如果没有重写过__eq__方法，就没用

False
True


### 特殊方法__hash__
用于实现根据对象生成hash值的逻辑

set和dict会用到

In [42]:
class MyDate:
    def __init__(self,year,month,day):
        self.year = year
        self.month = month
        self.day = day
    def __hash__(self):
        print("hash is called")
        return hash(self.year+self.month*101+self.day*101)
    
my_date1 = MyDate(2022,11,3)
data_set = set()
data_set.add(my_date1)


hash is called


### 特殊方法__bool__
用于在对象被bool函数求解的时候返回一个布尔值

在对象放在条件判断语句时用到

如果类没有实现这个方法，那么__len__将会被用户求解布尔值

In [43]:
class MyDate:
    def __init__(self,year,month,day):
        self.year = year
        self.month = month
        self.day = day
    def __bool__(self):
        print("bool is called")
        return self.year>2021
my_date1 = MyDate(2022,11,3)
my_date2 = MyDate(2021,11,3)
if my_date1:
    print('>2021')
if not my_date2:
    print('<=2021')


bool is called
>2021
bool is called
<=2021


### 特殊方法__del__
__del__方法在对象被垃圾回收前调用

因为对象何时被回收未知，所以不要依赖于这个方法去做一些重要的事情

In [44]:
class MyDate:
    def __init__(self,year,month,day):
        self.year = year
        self.month = month
        self.day = day
    def __del__(self):
        print("del is called")
my_date5 = MyDate(2024,11,3)
my_date5 = None # 不能保证是在这里被回收

del is called


## property类
如何防止实例变量被外部错误修改

编写setter和getter方法

引入property类

### 如何防止实例变量被外部错误修改

In [46]:
class Student:
    def __init__(self,name,age):
        self.name = name
        self.age = age

    def __str__(self):
        return f"{self.name} {self.age}"
student = Student("Jack",18)
student.age = -3    # 错误的代码不会提示
print(student)

Jack -3


如何避免？

In [48]:
class Student:
    def __init__(self,name,age):
        self.name = name
        self.__age = age
    def set_age(self, age):
        if age<0 or age>200:
            raise Exception(f"Age {age} is not valid")
        self.__age = age
    def get_age(self):
        return self.__age
    def __str__(self):
        return f"{self.name} {self.age}"
student = Student("Jack",18)
# student.age = -3    # 错误的代码不会提示
student.set_age(-3)    # 错误的代码会提示

Exception: Age -3 is not valid

问题是，如果之前的人已经用了self.age来修改，直接将age变为私有会不兼容

### 引入property类

In [57]:
class Student:
    def __init__(self,name,age):
        self.name = name
        self.__age = age
    def set_age(self, age):
        if age<0 or age>200:
            raise Exception(f"Age {age} is not valid")
        self.__age = age
    def get_age(self): 
        print('get_age')
        return self.__age
    def __str__(self):
        return f"{self.name} {self.age}"
    
    age = property(fget=get_age,fset=set_age)   # 将age设为property类，会绑定上get和set方法

student = Student("Jack",18)
student.age = -3    # 会调用set_age方法，报错会从set_age里面抛出


Exception: Age -3 is not valid

In [56]:
student = Student("Jack",18)
print(student.age)  # 会调用get_age方法

get_age
18


# property装饰器
property类的替代，更加优雅

In [69]:
class Student:
    def __init__(self,name,age):
        self.name = name
        self.__age = age

    @property
    def age(self): 
        return self.__age
    
    @age.setter # 方法名和上面的@property一致
    def age(self, age):
        if age<0 or age>200:
            raise Exception(f"Age {age} is not valid")
        self.__age = age

    def __str__(self):
        return f"{self.name} {self.age}"
    
student = Student("Jack",18)
print(student.age)
student.age =3 
print(student.age)
student.age =-3 
print(student.age)


18
3


Exception: Age -3 is not valid

## 只读property

最佳实践

In [71]:
class Square:
    def __init__(self,width):
        self.__width = width
    
    @property   # 让外部以为有这个属性
    def area(self):
        return self.__width*self.__width

square = Square(5)
print(square.area)

25


但每次执行都要重新计算，会有点浪费。能否缓存？

In [72]:
class Square:
    def __init__(self,width):
        self.__width = width
        self.__area = None
    
    @property   # 让外部以为有这个属性，第二次不用再计算
    def area(self):
        if self.__area is None:
            self.__area = self.__width*self.__width
        return self.__area

square = Square(5)
print(square.area)

25


但如果中途改了width，area会不准

下面这样改

In [73]:
class Square:
    def __init__(self,width):
        self.__width = width
        self.__area = None
    
    @property
    def area(self):
        if self.__area is None:
            self.__area = self.__width*self.__width
        return self.__area
    
    @property
    def width(self,width):
        return self.__width
    
    @width.setter
    def width(self, width):
        self.__width = width
        self.__area = None  #清空

square = Square(5)
print(square.area)
square.width = 6
print(square.area)


25
36


## 删除property

@属性名.deleter装饰器

In [75]:
class Square:
    def __init__(self,width):
        self.__width = width
        self.__area = None
    
    @property
    def area(self):
        if self.__area is None:
            self.__area = self.__width*self.__width
        return self.__area
    
    @property
    def width(self,width):
        return self.__width
    
    @width.setter
    def width(self, width):
        self.__width = width
        # self.__area = None  #清空

    @area.deleter
    def area(self):
        self.__area = None
square = Square(5)
print(square.area)
square.width = 6
del square.area # 会执行area.deleter方法
print(square.area)

25
36


## 类的继承

isinstance()

issubclass()

In [85]:
class Person:
    def __init__(self):
        print('Person:init is called')
        self.name = "Jack"

class Student(Person):  # 不写__init__，会自动调用父类的
    pass

student = Student()
print(student.name)

class Student(Person):
    def __init__(self):    # 重写__init__方法后，父类的__init__方法失效
        print('Student:init is called')
        self.school = 'ABC'

student = Student()
print(student.name)


Person:init is called
Jack
Student:init is called


AttributeError: 'Student' object has no attribute 'name'

In [87]:
class Student(Person):
    def __init__(self):
        super().__init__()  # 先执行父类的初始化
        self.school = 'ABC'
student = Student()
print(student.name)

Person:init is called
Jack


In [88]:
class Person:
    def __init__(self,name):
        self.name = name

class Student(Person):
    def __init__(self,name,school):
        super().__init__(name)
        self.school = school
student = Student('Jack','ABC')
print(student.name)
print(student.school)

Jack
ABC


isinstance()判断对象是否是一个类的对象或者该类的子类的对象

In [92]:
student = Student('Jack','ABC')
print(isinstance(student,Student))
print(isinstance(student,Person))
person = Person('Mike')
print(isinstance(person,Student))

True
True
False


issubclass()判断一个类是否为某个类的子类

In [93]:
issubclass(Student,Person)

True

# 方法重写

在子类中重新定义父类的方法

In [3]:
class Person:
    def __init__(self,name):
        self.name = name
    def say(self):
        print("Hello from person")
class Student(Person):
    def __init__(self,name,school):
        super().__init__(name)
        self.school = school
student = Student('Jack','ABC')
student.say()

Hello from person


In [4]:
class Student(Person):
    def __init__(self,name,school):
        super().__init__(name)
        self.school = school
    def say(self):
        print("Hello from student")
student = Student('Jack','ABC')
student.say()

Hello from student


In [5]:
def render(person: Person):
    person.say()

render(student) # 鸭子类型的表现。多态一种形式。不管对象属于哪个类，也不管声明的具体接口是什么，只要对象实现了相应的方法，函数就可以在对象上执行操作

Hello from student


In [6]:
class Worker(Person):
    pass
worker = Worker('Jack')
render(worker)  # 调用的是Person类中的say方法

Hello from person


### 类属性的覆盖

In [8]:
class Person:
    color = 1
    def __init__(self,name):
        self.name = name
    def say(self):
        print("Hello from person")
    def print_color(self):
        print(self.color)
class Student(Person):
    color = 3
    def __init__(self,name,school):
        super().__init__(name)
        self.school = school

student = Student('Jack','ABC')
student.print_color()   # 这里打印出来的是已经被覆盖的属性

3


### 在父类方法基础上再加点内容

In [9]:
class Person:
    def __init__(self,name):
        self.name = name
    def say(self):
        print("Hello from person")
class Student(Person):
    def __init__(self,name,school):
        super().__init__(name)
        self.school = school
    def say(self):
        super().say()   # 用super()调用父类的方法
        print('Hello from student')
student = Student('Jack','ABC')
student.say()

Hello from person
Hello from student


## 抽象类

抽象类是一个不能被实例化的类

抽象方法是一个没有具体实现的方法

一个抽象类可以有或者没有抽象方法

Python并没有直接支持抽象类，但是提供了一个模块（abc）来允许定义抽象类

### 如何定义抽象类
通过继承abc.ABC类定义一个抽象类

In [13]:
from abc import ABC
class Action(ABC):
    pass
action = Action()   # 没有抽象方法的抽象类是可以构造出对象的

In [14]:
from abc import ABC, abstractmethod
class Action(ABC):
    @abstractmethod # 抽象方法，不需要实现
    def execute(self):
        pass
action = Action()   # 会报错

TypeError: Can't instantiate abstract class Action with abstract method execute

In [15]:
class CreateStudentAction(Action):
    def execute(self):  # 得实现父类中的抽象方法
        print("Create a new student")

class DeleteStudentAction(Action):
    def execute(self):  # 得实现父类中的抽象方法
        print("Delete a student")

def execute_action(action: Action):
    action.execute()

create_student_action = CreateStudentAction()
delete_student_action = DeleteStudentAction()
execute_action(create_student_action)   # 传不同的对象会调用不同的方法
execute_action(delete_student_action)   # 传不同的对象会调用不同的方法

Create a new student


## 枚举

代码中的数值不利于人来阅读的时候，总希望能够有一个既方便阅读又利于查错的方法

### 如何定义枚举
通过继承enum.Enum类定义一个枚举类型

In [19]:
from enum import Enum
class Gender(Enum):
    MALE=1  # 成员1
    FEMALE=2    # 成员2
class Student:
    def __init__(self):
        self.gender = Gender.MALE
print(type(Gender.MALE))
print(Gender.MALE.name)
print(Gender.MALE.value)

<enum 'Gender'>
MALE
1
MALE 1
FEMALE 2


### 存数据库怎么办

#### 字符串转枚举

In [20]:
s_gender= "MALE"    # 假设从数据库中取出字符串形式的值
# 怎么变成枚举？
student = Student()
student.gender = Gender[s_gender]   # 将字符串转为枚举类型
print(student.gender)

Gender.MALE


#### 数字转枚举

In [21]:
s_gender= 1   # 假设从数据库中取出数字形式的值
# 怎么变成枚举？
student = Student()
student.gender = Gender(s_gender)   # 将数字转为枚举类型
print(student.gender)

Gender.MALE


### 遍历枚举
枚举本身是可迭代的

In [22]:
for gg in Gender:
    print(gg)

Gender.MALE
Gender.FEMALE


### 枚举的继承
枚举可以继承其他没有成员的枚举，有成员的枚举不能被继承

## 枚举别名和装饰器

### 枚举成员的别名
当枚举中的多个成员具有相同value的时候，只有一个能成为主要的成员，其他的都自动成为别名

In [26]:
class Status(Enum):
    SUCCESS = 1
    OK = 1
    FAIL = 2
    WRONG = 2
for i in Status:
    print(i.name)   # 同value值中第一次出现的name才能被遍历到，即主要成员
print(Status.__members__)   # __members__记录了全部的name
print(Status.SUCCESS == Status.OK)
print(Status.SUCCESS is Status.OK)  # OK没有指向新的对象

SUCCESS
FAIL
{'SUCCESS': <Status.SUCCESS: 1>, 'OK': <Status.SUCCESS: 1>, 'FAIL': <Status.FAIL: 2>, 'WRONG': <Status.FAIL: 2>}
True
True


### @enum.unique装饰器
当我们需要保证枚举中的成员必须具有唯一值的时候，可以使用@enum.unique来约束

In [28]:
import enum
@enum.unique
class Gender(Enum):
    MALE=1
    FEMALE=1    # 加了unique装饰器后直接就会报错

ValueError: duplicate values found in <enum 'Gender'>: FEMALE -> MALE

## 定制和扩展枚举

### 定制__str__函数

In [29]:
class Status(Enum):
    SUCCESS=1
    FAIL=2
    def __str__(self):
        return f"{self.name}({self.value})"
for s in Status:
    print(s)

SUCCESS(1)
FAIL(2)


### 定制__eq__函数

In [30]:
class Status(Enum):
    SUCCESS=1
    FAIL=2
    def __str__(self):
        return f"{self.name}({self.value})"
    def __eq__(self,other):
        if isinstance(other,int):
            return self.value==other
        if isinstance(other,str):
            return self.name==other.upper()
        if isinstance(other,Gender):
            return self is other
        return False
        
print(Status.SUCCESS==1)
print(Status.SUCCESS=="SUCCESS")
print(Status.SUCCESS=="FAILED")

True
True
False


### 定制__lt__函数

In [32]:
from functools import total_ordering

@total_ordering
class WorkFlowStatus(Enum):
    OPEN=1
    IN_PROGRESS=2
    REVIEW=3
    COMPLETE=4
    def __lt__(self,other):
        if isinstance(other,int):
            return self.value<other
        if isinstance(other,WorkFlowStatus):
            return self.value<other.value
        return False

print(WorkFlowStatus.IN_PROGRESS<2)
print(WorkFlowStatus.IN_PROGRESS<WorkFlowStatus.COMPLETE)


False
True


### auto()函数
当希望程序自动按顺序来给枚举成员赋值的时候，可以使用auto()函数。不推荐用

In [ ]:
from enum import auto


class Status(Enum):
    SUCCESS=auto()
    FAILED=auto()

## 多继承
### 多继承的语法

In [33]:
class Parent1:
    def render(self):
        print("parent 1")
class Parent2:
    def hello(self):
        print("hello parent 2")
class Child(Parent1,Parent2):
    pass
child = Child()
child.render()
child.hello()


parent 1
hello parent 2


### 同名函数的继承问题

In [35]:
class Parent1:
    def render(self):
        print("parent 1")
class Parent2:
    def render(self):
        print("parent 2")
    def hello(self):
        print("hello parent 2")
class Child(Parent1,Parent2):
    pass
child = Child()
child.render()
class Child(Parent2,Parent1):
    pass
child = Child()
child.render()

parent 1
parent 2


以继承的书写顺序来确定，先继承那个就调谁的

如果希望手动选择继承哪个类的方法，怎么办

In [37]:
class Parent1:
    def render(self):
        print("parent 1")
    def hello(self):
        print("hello parent 1")

class Parent2:
    def render(self):
        print("parent 2")
    def hello(self):
        print("hello parent 2")

In [38]:
class Child(Parent2,Parent1):   # 希望执行Parent1的hello
    def hello(self):
        Parent1.hello(self) # 重点，将self传入
child = Child()
child.render()
child.hello()

parent 2
hello parent 1


尽量使用单继承

## 描述符

### 定义属性时的麻烦

In [39]:
class Student:
    def __init__(self,first_name,last_name):
        self.first_name = first_name
        self.last_name = last_name
    @property
    def first_name(self):
        return self.__first_name
    @first_name.setter
    def first_name(self,first_name):
        if not isinstance(first_name,str):
            raise Exception("First name is not a string")
        if len(first_name)==0:
            raise Exception("First name is empty")
        self.__first_name = first_name
    @property
    def last_name(self):
        return self.__last_name
    @last_name.setter
    def last_name(self,last_name):
        if not isinstance(last_name,str):
            raise Exception("Last name is not a string")
        if len(last_name)==0:
            raise Exception("Last name is empty")
        self.__last_name = last_name
student = Student('','')

Exception: First name is empty

每个属性校验都要写一遍代码，非常麻烦

#### 描述符的规定
描述符是一个特殊的类，要求实现如下一些方法
* 方法__set_name__(self,owner,name)
* 方法__get__(self,instance,owner)
* 方法__set__(self,instance,value)
* 方法__delete__(self,instance)

In [46]:
class RequiredString:   # 可复用到所有的类
    def __init__(self,trim:bool=False):
        self.__trim = trim

    def __set_name__(self,owner,name):
        self.__property_name = name

    def __set__(self,instance,value):
        if not isinstance(value,str):
            raise Exception(f"{self.__property_name} is not a string")
        if self.__trim:
            value = value.strip()
        if len(value)==0:
            raise Exception(f"{self.__property_name} is empty")
        instance.__dict__[self.__property_name] = value
    
    def __get__(self,instance,owner):
        if self.__property_name in instance.__dict__:
            return instance.__dict__[self.__property_name]

class Student2:
    first_name = RequiredString(True)    # 用描述符校验，并将类变量变成属性
    last_name = RequiredString()

student2 = Student2()
student2.first_name = 'Jack'    # 调用类变量，类变量指向修饰符对象，因此会调用__set__方法。即first_name.__set__(student2,"Jack")
student2.last_name = 'Johnson'

student3=Student2()
student3.first_name = ' Tom'
# student3.last_name = ''
print(student2.first_name)
print(student3.first_name)

Jack
Tom


## 特殊方法__new__方法

### 方法__new__的定义原型
方法__new__定义在object类中，所有类的最终父类都是object类
* `object.__new__(cls,*args,**kwargs)`

构造一个对象的过程，比如:
`person=Person("Jack")`

1. `person=object.__new__(Person,"Jack")`
2. `person.__init__("Jack")`

### `__new__`方法的最佳实践
通常情况下定义了__new__就不用再定义__init__方法了

In [47]:
class SquareNumber(int):
    def __new__(cls,value:int):
        return super().__new__(cls,value**2)
num = SquareNumber(2)
print(num)
print(type(num))
print(isinstance(num,int))

4
<class '__main__.SquareNumber'>
True


In [49]:
class Student:
    def __new__(cls,first_name,last_name):
        obj = super().__new__(cls)
        obj.first_name = first_name
        obj.last_name = last_name
        return obj
student = Student("Jack","Ma")
print(student.first_name)
print(student.last_name)

Jack
Ma


# 异常类
* 常用的异常类继承关系
* 异常类的捕捉
* 同时处理多个异常
* raise异常
* 自定义异常

### 常用的异常类继承关系
![](../images/异常类.png)

红色是系统级别故障，蓝色是程序级别异常

### 异常类的捕捉
try ... except语句，可以使用特定的的异常类来捕捉，也可以用父类型的异常类来捕捉

### 同时处理多个异常
在同时捕捉多个异常的时候，需要先捕捉子类型的异常，后捕捉父类型的异常

In [55]:
try:
    a = 5/0
except AssertionError:
    print('AssertionError')
except ArithmeticError:
    print('ArithmeticError')
except ZeroDivisionError:
    print('除数不能为0')    # 前面的父类已经找到了就不会到这
except Exception:   # 所有上面没有被捕获的异常类型
    print('All Exception')

All Exception


### raise异常
可以在代码中通过raise异常的方式来向调用者返回错误信息

In [59]:
def add(n1,n2):
    if not isinstance(n1,int) or not isinstance(n2,int):
        raise Exception("Argument is not int")
    return n1+n2
try:
    add('a',2)
except Exception as ex: # 可以获取到抛出的异常对象
    print(f'Exception: {ex}')

Exception: Argument is not int


### 自定义异常

In [61]:
class InvalidArgumentException(Exception):  # 自定义异常
    def __init__(self,*args,**kwargs):
        super().__init__(args)


def add(n1,n2):
    if not isinstance(n1,int) or not isinstance(n2,int):
        raise InvalidArgumentException("Argument is not int")
    return n1+n2
try:
    add('a',2)
except InvalidArgumentException as ex: # 可以获取到抛出的异常对象
    print(f'InvalidArgumentException: {ex}')

InvalidArgumentException: ('Argument is not int',)
